In [0]:
WITH exploded AS (
    SELECT
        mac_addr,
        sales_model_code,
        country_code,
        platform_version,
        hdmi_item.hdmi_label
    FROM aic_data_mart.tv_usage.hdmi_usage
    LATERAL VIEW explode(hdmi_info) t AS hdmi_item
    WHERE date_ym = '2023-12'
      AND platform_version = 'webOS23'
      AND hdmi_item.hdmi_type = 'audio'
),
total_audio_mac AS (
    SELECT COUNT(DISTINCT mac_addr) AS total_audio_unique_mac
    FROM exploded
)
SELECT
    platform_version,
    country_code,
    sales_model_code,
    hdmi_label,
    COUNT(DISTINCT mac_addr) AS unique_mac_count,
    total_audio_mac.total_audio_unique_mac AS total_audio_mac_count,
    ROUND(COUNT(DISTINCT mac_addr) / total_audio_mac.total_audio_unique_mac * 100, 2) AS percentage
FROM exploded
CROSS JOIN total_audio_mac
GROUP BY platform_version, country_code, sales_model_code, hdmi_label, total_audio_mac.total_audio_unique_mac
ORDER BY unique_mac_count DESC;